# ConversationSummaryBufferMemory

`ConversationSummaryBufferMemory` は、前回の2つのアイデアを組み合わせたものです。最近の対話のバッファをメモリに保持し、古い対話を完全に消去するのではなく、サマリーとして編集し、その両方を使用します。ただし、前回の実装とは異なり、対話のフラッシュを決定するためにトークンの長さを使用します。

まず、ユーティリティの使用方法を説明します。

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import OpenAI
llm = OpenAI()

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "やあ"}, {"output": "どうしたの？"})
memory.save_context({"input": "なにもない"}, {"output": "あまりない"})

In [ ]:
memory.load_memory_variables({})

履歴をメッセージのリストとして取得することもできます (これはチャットモデルと一緒に使用する場合に便利です)。

In [ ]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=300, return_messages=True)
memory.save_context({"input": "やあ"}, {"output": "どうしたの？"})
memory.save_context({"input": "なんでもない"}, {"output": "なんだ"})

また、`predict_new_summary`メソッドを直接利用することも可能です。

In [ ]:
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

## チェーンで使う
プロンプトが表示されるように `verbose=True` を設定して、サンプルを見てみましょう。

In [ ]:
from langchain.chains import ConversationChain
conversation_with_summary = ConversationChain(
    llm=llm, 
    # テストのため、非常に低いmax_token_limitを設定しました。
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=40),
    verbose=True
)
conversation_with_summary.predict(input="やあ、どうしたんだい？")

In [ ]:
conversation_with_summary.predict(input="ただ、ドキュメントを書く作業をしています！")

In [ ]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="LangChain（ラングチェーン）について！聞いたことありますか？")

In [ ]:
# We can see here that the summary and the buffer are updated
conversation_with_summary.predict(input="いや、そうではなく、多くの人がそれを混同しているのです。")